<a href="https://colab.research.google.com/github/iSANDEx/fB_DCEMRI_analyser/blob/main/jupyter/GoogleColab_RegistrationWorkflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (TODO) Notebook Title and description

# Setting up the running environment
These set of cells will prepare the environment to run the registration workflow

## Installation and Loading Modules
It installs specific tools to manage medical images and DICOM data:
* [ITK](https://github.com/InsightSoftwareConsortium/ITKPythonPackage)
* [ANTs](https://github.com/ANTsX/ANTsPy)
* [PyDICOM](https://pydicom.github.io/pydicom/stable/index.html)
* [DICOM2NIFTI](https://github.com/icometrix/dicom2nifti)
and also, it install additional tools for visualisation and display:
* [directory_tree](https://github.com/rahulbordoloi/Directory-Tree/)


In [55]:
!pip install pydicom dicom2nifti directory_tree itk ants itk-elastix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 26.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of itk to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 23.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of itk to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 14.3 MB/s eta 0:00:00


In [35]:
import os
import itk
import sys
import math
import time
import ants
import json
import glob
import shutil
import numpy as np
import dicom2nifti
import pydicom as pyd
from datetime import timedelta
from directory_tree import display_tree  # Nice tool to display directory trees (https://pypi.org/project/directory-tree/)



## Auxiliar Functions

In [32]:
def list_folder_content(path, show_hidden=False):
    if show_hidden:
        ddfldrlst = os.listdir(path)
    else:
        ddfldrlst = list(filter(lambda item: not item.startswith('.'),os.listdir(path)))
    return ddfldrlst

def display_folder_list(file_list):
    print('\n'.join(f'[{idx}] - {file_idx}' for idx, file_idx in enumerate(file_list)))

def get_path_to_process(full_path):
    print('Folder content:')
    print(display_tree(full_path, header=True, string_rep=True, show_hidden=False, max_depth=2))
    folder_content = list_folder_content(full_path)
    # Ideally we'll have only one sub-folder inside the PreTreatment folder. If more than one, then we have to choose, but by default, we'll select the first one.
    idx_reg = 0
    if len(folder_content) > 1:
        display_folder_list(folder_content)
        idx_sel = input(f'Select the folder with the dataset_to_process to process (0-{len(folder_content)-1} or just press Enter to proceed with sub-folder {folder_content[idx_reg]}):')
        if idx_sel:
            idx_reg = int(idx_sel)
    path2data = os.path.join(full_path, folder_content[idx_reg])
    print(f'Will process {folder_content[idx_reg]}')
    return path2data

def check_time_points(path_to_check, nmax = 6, verbose=False):
    if path_to_check is not None:
        nr_of_folders = list_folder_content(path_to_check)
        print(f'Folder {path_to_check} seems Ok' if len(nr_of_folders)== nmax else f'Error! Check path {path_to_check} is the correct one')
        if verbose:
            print('Listing folder content:')
            display_tree(path_to_check, max_depth=1)
        return nr_of_folders if len(nr_of_folders) == nmax else None
    else:
        return None

def add_prefix_to_filename(full_path, prefix=None):
    # Assume the last part of the path is the filename (with extension)
    file_path, file_name_ext = os.path.split(full_path)
    if prefix:
        updated_filename = '_'.join([prefix, file_name_ext])
        return os.path.join(file_path, updated_filename)
    else:
        return prefix

def itk2ants(itkInput):
    antsOutput = ants.from_numpy(itk.GetArrayFromImage(itkInput).T,
                                 origin=tuple(itkInput.GetOrigin()),
                                 spacing=tuple(itkInput.GetSpacing()),
                                 direction=np.array(itkInput.GetDirection()))

    return antsOutput

def ants2itk(antsInput):
    itkOutput = itk.GetImageFromArray(antsInput.numpy().T)
    itkOutput.SetOrigin(antsInput.origin)
    itkOutput.SetSpacing(antsInput.spacing)
    itkOutput.SetDirection(antsInput.direction)

    return itkOutput

def getenv(colab=False):
    """
    Requires sys and os modules:
    import sys
    import os
    Possible values for sys.platform are (https://docs.python.org/3/library/sys.html & https://stackoverflow.com/questions/446209/possible-values-from-sys-platform)
    ┍━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┑
    │  System             │ Value               │
    ┝━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━┥
    │ Linux               │ linux or linux2 (*) │
    │ Windows             │ win32               │
    │ Windows/Cygwin      │ cygwin              │
    │ Windows/MSYS2       │ msys                │
    │ Mac OS X            │ darwin              │
    │ OS/2                │ os2                 │
    │ OS/2 EMX            │ os2emx              │
    │ RiscOS              │ riscos              │
    │ AtheOS              │ atheos              │
    │ FreeBSD 7           │ freebsd7            │
    │ FreeBSD 8           │ freebsd8            │
    │ FreeBSD N           │ freebsdN            │
    │ OpenBSD 6           │ openbsd6            │
    │ AIX                 │ aix (**)            │
    ┕━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┙
    Update 03/04/2024: Added support when running inside Google Colab Environment, via the optional flag "colab"
    """
    if colab:
      # Requires mounting the Google Drive before
      HOMEPATH = '/content/drive/MyDrive'
    else:
      if sys.platform == 'win32':
          env_home = 'HOMEPATH'
      elif (sys.platform == 'darwin') | (sys.platform == 'linux'):
          env_home = 'HOME'
      HOMEPATH = os.getenv(env_home)

    return HOMEPATH

def check_path_exist(path, file=False):
    """
    Flag FILE indicates the path contains a file name (FLAG=TRUE) or the path only points to a folder (FLAG=FALSE (Default))
    """
    if file:
        is_path = os.path.isfile(path)
    else:
        is_path = os.path.isdir(path)

    print(f'{"OK:" if is_path else "ERROR:"} Path to {"file" if file else "folder"} {path} does{"" if is_path else " NOT"} exist')

    return is_path

def create_folder_structure(root_path, sub_dirs, create=True):
  """
  Create subdirectories insode HOMEPATH (using makedirs to avoid altering existing folders)
  ROOT_PATH must exist, otherwise it issues an error
  SUB_DIRS is a dictionary with the subfolder required inside HOMEPATH

  """
  subdirpaths = {}
  for folder_name, sub_dir in sub_dirs.items():
    path_to_subdir = os.path.join(root_path, sub_dir)
    if create:
        os.makedirs(path_to_subdir, exist_ok=True)
        subdirpaths[folder_name] = path_to_subdir


  return subdirpaths



## Path to Data and Google Drive
Mount the user-owned Google Drive to access the DICOM data. It requires access through the google account, but none of the access credentials are stored in the notebook.

In [14]:
# Mount the user-owned Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# Create the Folder Structure (slightly different to the one created for testing registration methods, but still follows the same criteria)
HOMEPATH = getenv(colab=True)
ROOTPATH = os.path.join(HOMEPATH, 'Data', 'fMRIBreastData')

SUBDIRFLDRS = {'STUDYFLDR':   'StudyData',
               'NIFTIFLDR':   'NiftiData',
               'SRCDCMFLDR':  'rawS3',
               'REGOUTFLDR':  'RegOutput',
               'CFGFLDR':     'configFiles',
               'OUTPUTFLDR':  'output',
               'LOGFLDR':     'logs'
              }
subdir_paths = create_folder_structure(ROOTPATH, SUBDIRFLDRS)

# DICOM Sources:
srcpath = subdir_paths['SRCDCMFLDR']
dstpath = subdir_paths['STUDYFLDR']
# Check the path exist and are correct:
# check_path_exist(srcpath)
# check_path_exist(dstpath)
fext = 'dcm'

Now that the folders have been created, (manually) place the dicom (raw) data into the ```rawS3``` folder (or whatever value is in ```subdir_paths['SRCDCMFLDR']```)

After done that, we'll sort them out automatically in the ```subdir_paths['STUDYFLDR']```

# Sort out DICOM folders
Sort out dicom files following this order:
* PatientID () --> Not used
* PatientName () --> Use this as it is also used by Radiologists, so to keep consistency
* StudyID () --> Not needed
* StudyDate () --> This defines the pre- and post-treatment (just the earliest is pre- and the latest is post-)
* SeriesNro () --> This is relevant to later eliminate any possible in-scanner post-processed data
* SeriesDescription ()


In [ ]:
# Get the list of dicom files in the folder, use the extension defined by FEXT:
dcmlist = glob.glob(os.path.join(srcpath,f'*.{fext}'))
print(f'There are {len(dcmlist)} files to process in {srcpath}. Please wait...')

start_time = time.perf_counter()

nels = len(dcmlist)
nsteps = 10**math.floor(math.log10(0.01*nels)+1)

print(''.join(['*']*100))
for nimg, dcm in enumerate(dcmlist):
    if (nimg % nsteps)==0:
        print(f'\t{nels-nimg} files to process...')

    ds = pyd.dcmread(dcm,stop_before_pixels=True)
    [PatientID,
     PatientName,
     StudyDate,
     SeriesNro,
     TempPos] = [ds.PatientID, str(ds.PatientName), ds.StudyDate,
                 str(ds.SeriesNumber), str(ds.TemporalPositionIdentifier)]

    name_as_list = PatientName.split(' ')
    # remove multiple spaces:
    name_no_space = [i for i in name_as_list if i != '']
    # From the second element onward, use camel-case:
    name_camel_case = [i.title().replace('Treatmensst','Treatment') if idx>0 else i for idx, i in enumerate(name_no_space)]
    # Re-Join the name with a dash instead of (multiple) spaces:
    PatientName = '-'.join(name_camel_case)
    folderStruct = os.path.join(dstpath,
                                '-'.join([PatientName[:PatientName.find('-')],
                                          PatientID]),
                                '-'.join([PatientName.replace(' ','_'),
                                          StudyDate.replace(' ','_')]),
                                SeriesNro.replace(' ','_'),
                                TempPos.replace(' ','_'))
    os.makedirs(folderStruct, exist_ok=True)
    dstFile = os.path.join(folderStruct, os.path.split(dcm)[-1])
    if not os.path.isfile(dstFile):
        shutil.copy2(dcm, dstFile)
    else:
        print(f'File {dstFile}, already exists. Nothing done')
end_time = time.perf_counter()
elp_time = end_time - start_time
print(''.join(['*']*100))
print(f'All done (Elapsed time was {elp_time:.1f}[s]). Showing just the last folder processed:')
print(f'{folderStruct}')


# Convert DICOM to NIFTI

In [47]:
dcmpath = subdir_paths['STUDYFLDR']
niftipath = subdir_paths['NIFTIFLDR']
# Check the path exist and are correct:
# check_path_exist(dcmpath)
# check_path_exist(niftipath)

# Displays the content of the DICOM folders that will be converted to NIFTI:
print(display_tree(dcmpath, header=True, string_rep=True, show_hidden=False, max_depth=4))

StudyData/
└── DC-ANON97378/
    ├── DC-Post-Treatment-20230726/
    │   └── 301/
    │       ├── 1/
    │       ├── 2/
    │       ├── 3/
    │       ├── 4/
    │       ├── 5/
    │       └── 6/
    └── DC-Pre-Treatment-20230621/
        └── 301/
            ├── 1/
            ├── 2/
            ├── 3/
            ├── 4/
            ├── 5/
            └── 6/



In [48]:
# Expected sub-folder tree follows the pattern:
# PATIENT_INITIALS+"-"+PATIENTID -> PATIENT_NAME+"-"+DATE_OF_VISIT -> DCE_MRI_SEQUENCE
# Valid PATIENT_NAMES contains at most 3 dashes (i.e. to exclude "Motion Corrected" one, but keeping "RICE001")
#
start_time = time.perf_counter()
concat_vol = False
PATIENTSID = list_folder_content(dcmpath)

print(''.join(['*']*100))
for patientID in PATIENTSID:
    path2patientID = os.path.join(dcmpath, patientID)
    patient_visits = list_folder_content(path2patientID)
    for visit_name in patient_visits:
        path2visit = os.path.join(path2patientID, visit_name)
        if visit_name.count('-') > 3:
            print(f'Folder Name is too long to match the criteria. Skipping {path2visit}')
            continue
        print(f'Processing {visit_name}. Please wait...')
        seq_nro = os.path.join(path2visit, list_folder_content(path2visit)[0])
        dce_time_points = list_folder_content(seq_nro)
        if concat_vol:
            tseries_volume = [None]*len(dce_time_points)
        for time_point_i in dce_time_points:
            path2time_point = os.path.join(seq_nro, time_point_i)
            print(f'Replicating sub-folder "{time_point_i}" in the output directory {niftipath}:')
            ipath = path2time_point
            opath = ipath.replace(dcmpath, niftipath)
            print(f'Creating folder {opath}...')
            try:
                os.makedirs(opath, exist_ok=True)
            except Exception as err:
                print(f'ERROR: Cannot create the folder {opath}')
                break
            print(f'Performing conversion from {ipath}, please wait...')
            dicom2nifti.convert_directory(ipath, opath)
            # if concat_vol:
            #     tseries_volume[int(time_point_i)-1] = itk.imread(glob.glob(os.path.join(opath,'*.nii.gz'))[0])
        if concat_vol:
            print(f"Conversion done successfully, now will concatenate the timeseries into a single 4D volume saved at {os.path.join(path2visit.replace(INPUTPATH, OUTPUTPATH),'.'.join([visit_name,'nii.gz']))}. Please wait a little bit more :) ...")
            # Initialise the 4d volume with the first time point (pre-contrast)
            t0_nii = tseries_volume[0]
            in_dim = t0_nii.GetImageDimension()
            pixel_type = itk.template(t0_nii)[1][0]
            out_dim = in_dim + 1

            input_image_type = itk.Image[pixel_type, in_dim]
            output_image_type = itk.Image[pixel_type, out_dim]

            layout = [1, 1, 1, len(dce_time_points)]
            vol_tiles = itk.TileImageFilter[input_image_type, output_image_type].New()
            vol_tiles.SetLayout(layout)
            for idx in range(len(dce_time_points)):
                vol_tiles.SetInput(idx, tseries_volume[idx])
            # Write 4D Volume:
            volume_writer = itk.ImageFileWriter[output_image_type].New()
            volume_writer.SetFileName(os.path.join(path2visit.replace(dcmpath, niftipath),'.'.join([visit_name,'nii.gz'])))
            volume_writer.SetInput(vol_tiles.GetOutput())
            volume_writer.Update()
            # And removing useless data:
            print(f'Removing {seq_nro} and its content...')
            shutil.rmtree(seq_nro.replace(dcmpath, niftipath))
        else:
            print('Conversion done successfully, moving to the next data folder.')
        print(''.join(['*']*100))

end_time = time.perf_counter()
elp_time = end_time - start_time

print(f'All done (Elapsed time was {elp_time:.1f}[s]), converted files have been saved at {dcmpath}. Bye!')

Processing DC-Pre-Treatment-20230621. Please wait...
Replicating sub-folder "3" in the output directory /content/drive/MyDrive/Data/fMRIBreastData/NiftiData:
Creating folder /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/3...
Performing conversion from /content/drive/MyDrive/Data/fMRIBreastData/StudyData/DC-ANON97378/DC-Pre-Treatment-20230621/301/3, please wait...
Replicating sub-folder "6" in the output directory /content/drive/MyDrive/Data/fMRIBreastData/NiftiData:
Creating folder /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/6...
Performing conversion from /content/drive/MyDrive/Data/fMRIBreastData/StudyData/DC-ANON97378/DC-Pre-Treatment-20230621/301/6, please wait...
Replicating sub-folder "5" in the output directory /content/drive/MyDrive/Data/fMRIBreastData/NiftiData:
Creating folder /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/5...

# Register the Nifti datasets
Here, we separate the process in functions (similar to Inter-visit registration)


In [51]:
savepath = subdir_paths['REGOUTFLDR']
studypath = subdir_paths['NIFTIFLDR']


config = subdir_paths['CFGFLDR']
logpath = subdir_paths['LOGFLDR']

# configpath = subdir_paths['CFGFLDR']
# Check the path exist and are correct:
# check_path_exist(dcmpath)
# check_path_exist(niftipath)


# default settings:
DEBUGMODE = True
BATCHMODE = False # TRUE: it runs the registration for all dataset within STUDYPATH; FALSE (DEFAULT): allows to pick a specific dataset to register

registration_algorithm = 'Elastix'
fixed_volume_pos = 2
register_fixed = True
config_files = ['Par0032_Elastix_Params_001_rigid.txt', 'Par0032_Elastix_Params_002_bsplines.txt']
txt_description = 'Default registration (Stage 1) of functional Breast DCE-MRI datasets'
platform = 'any'
bias_correction = 'n4itk'
histogram_matching = False


dataset_to_process = {'study_path': studypath,
                      'save_path': savepath,
                      'data_path': os.path.join(savepath, 'datasets'),
                      'parameters_folder': 'parameters',
                      'intended_platform': platform,
                      'run_platform': sys.platform,
                      'registration_details': {'algorithm': registration_algorithm,
                                               'configuration_files': config_files,
                                               'register_fixed': register_fixed},
                      'fixed_volume_position': fixed_volume_pos,
                      'preprocessing': {'bias_correction': bias_correction,
                                        'histogram_matching': histogram_matching},
                      'datasets': {}
                     }
# Create the top level folder(s) inside SAVEPATH:
os.makedirs(os.path.join(dataset_to_process['save_path'],dataset_to_process['parameters_folder']), exist_ok=True)

# Create description.json file from the descriptive variables defined in the previous cell:
description = {'Summary': txt_description,
               'Intended Platform': platform,
               'Run Platform': sys.platform,
               'Registration Details': {'algorithm': registration_algorithm,
                                        'configuration parameters': config_files,
                                        'reference volume': fixed_volume_pos,
                                        'register fixed': register_fixed},
               'Preprocessing': {'bias_correction': bias_correction,
                                 'histogram_matching': histogram_matching}
               }

# Save the description as a JSON file in the output directory:
with open(os.path.join(dataset_to_process['save_path'], 'description.json'), 'w') as fp:
    json.dump(description, fp)

In [52]:
# List the patients in the root folder:
patients = list_folder_content(studypath)
print('Patient data folders:')
display_folder_list(patients)
if not BATCHMODE:
    # # Pick up an option:
    # patients_indices = range(len(patients))
    # patientIDX = None
    # while patientIDX not in patients_indices:
    patientIDX = input(f'Pick up a valid index to select a patient {tuple(range(len(patients)))} or type "a" to process all:')
    if patientIDX == 'a':
        print(f'Will process all patients in the test folder')
    else:
        patientIDX = int(patientIDX)
        patients = [patients[patientIDX]]
        if DEBUGMODE:
            print(f'Patient {patients[0]} selected contains the follow datasets:')
            print(display_tree(os.path.join(studypath, patients[0]), header=True, string_rep=True, show_hidden=False, max_depth=4))
else:
    print(f'Processing the whole data folder {studypath} as a batch process \n***Please be patient!!***')

Patient data folders:
[0] - DC-ANON97378
Pick up a valid index to select a patient (0,) or type "a" to process all:a
Will process all patients in the test folder


It is expected the patient folder contains only 2 sub-folders:
* PatientName-Pre-Treatment-\<visit-date\>
* PatientName-Post-Treatment-\<visit-date\>

and inside each of these sub-folders, there is the sequence number and the timepoints, where the corresponding Nifti file lives:
```
    PatientName-<Pre/Post>-Treatment-<visit-date>/
    ├── SeqNro/
        ├── 1/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 2/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 3/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 4/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 5/
            ├── <SeqNro>_<sequence-name>.nii.gz
        ├── 6/
            ├── <SeqNro>_<sequence-name>.nii.gz
```
Note that for each timepoint, there is no difference in the Nifti file name, it is only differentiated by the folder enclosing it

In [53]:
# Organise the code to run for a single dataset and then, embed it into a loop for batch processing...
# dataset_to_process = {}
for patient in patients:
    # loop over patient datafolder (pre- and post-treatment)
    data_patient = os.path.join(studypath, patient)
    dataset_to_process['datasets'][patient]={'output_path': patient, # data_patient.replace(studypath, savepath),
                                 'visits': {}}
    patient_visits = list_folder_content(data_patient)
    checks = True
    if DEBUGMODE:
        print(f'Subfolders inside {data_patient}:\n\t{patient_visits}')
    print(''.join(['*']*50))
    for patient_visit in patient_visits:
        visit_name = patient_visit.split('-')
        seq_path = os.path.join(data_patient, patient_visit)
        seq_nro = list_folder_content(seq_path)[0]
        dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])] = {'path': os.path.join(patient_visit, seq_nro),
                                                                           'path2fixed': ''}
        # Check the number of subfolders and depth are the expected ones:
        print(f'Checking {seq_path} contains only 1 folder...')
        check_nsequences_per_visit = check_time_points(seq_path, nmax=1, verbose=DEBUGMODE)
        if check_nsequences_per_visit is not None:
            print(f'Checking {seq_nro} contains the expected number of timepoints ...')
            check_timepoints_per_seq = check_time_points(os.path.join(seq_path, seq_nro), verbose=DEBUGMODE)
        else:
            checks = False

        if check_timepoints_per_seq is not None:
            dce_tpoints_path = list_folder_content(os.path.join(seq_path, seq_nro))
            dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])]['path2moving'] = [None]*len(dce_tpoints_path)
            for time_point_i in dce_tpoints_path:
                print(f'Checking there is only one NIFTI file for each timepoint in {seq_nro}...')
                check_nfiles_per_timepoint = check_time_points(os.path.join(seq_path, seq_nro, time_point_i), nmax=1)
                get_nii_file = glob.glob(os.path.join(seq_path, seq_nro, time_point_i,'*.nii.gz'))
                if (check_nfiles_per_timepoint is not None) and (len(get_nii_file)==1):
                    print('Ready to load data...')
                    if int(time_point_i) == fixed_volume_pos:
                        dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])]['path2fixed'] = get_nii_file[0]
                    dataset_to_process['datasets'][patient]['visits'][''.join(visit_name[1:3])]['path2moving'][int(time_point_i)-1] = get_nii_file[0]
                else:
                    checks = False
        else:
            checks = False

        if not checks:
            print(f'***ERROR***: There is something wrong with the data in {seq_path}, please check!!')
        print(''.join(['*']*100))
    if checks:
        print(f'All done loading NIFTI files from dataset {data_patient}')
    else:
        print(f'***ERROR***: There is something wrong with some (or all) data in {data_patient}, please check!!')
    print(''.join(['*']*100))

if DEBUGMODE:
    print(f'Details of the datasets to process:')
    print(json.dumps(dataset_to_process, indent=1))

Subfolders inside /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378:
	['DC-Pre-Treatment-20230621', 'DC-Post-Treatment-20230726']
**************************************************
Checking /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621 contains only 1 folder...
Folder /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621 seems Ok
Listing folder content:
DC-Pre-Treatment-20230621/
└── 301/
Checking 301 contains the expected number of timepoints ...
Folder /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301 seems Ok
Listing folder content:
301/
├── 1/
├── 2/
├── 3/
├── 4/
├── 5/
└── 6/
Checking there is only one NIFTI file for each timepoint in 301...
Folder /content/drive/MyDrive/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/3 seems Ok
Ready to load data...
Checking there is only one NIFTI file for each timepoint in 30

In [54]:
# Registration parameters (this is the meat of the work!)
if dataset_to_process['registration_details']['algorithm'].lower() == 'ants':
    # ANTs
    # For details about possible values and description of parameters, see the help page: https://antspy.readthedocs.io/en/latest/registration.html
    # Default values (as listed in the hep page)
    dataset_to_process['par_set'] = {'type_of_transform': 'SyN',
                                     'initial_transform': None,
                                     'outprefix': '',
                                     'mask': None,
                                     'moving_mask': None,
                                     'mask_all_stages': False,
                                     'grad_step': 0.2,
                                     'flow_sigma': 3,
                                     'total_sigma': 0,
                                     'aff_metric': 'mattes',
                                     'aff_sampling': 32,
                                     'aff_random_sampling_rate': 0.2,
                                     'syn_metric': 'mattes',
                                     'syn_sampling': 32,
                                     'reg_iterations': (40, 20, 0),
                                     'aff_iterations': (2100, 1200, 1200, 10),
                                     'aff_shrink_factors': (6, 4, 2, 1),
                                     'aff_smoothing_sigmas': (3, 2, 1, 0),
                                     'write_composite_transform': False,
                                     'random_seed': None
                                    }

    # To ensure reproducibility of the results, set the random_seed to a constant value:
    dataset_to_process['par_set']['random_seed'] = 42 #(just to keep along with the pop-culture reference, e.g. https://medium.com/geekculture/the-story-behind-random-seed-42-in-machine-learning-b838c4ac290a
    # Save the parameters as a JSON file in the parameters folder:
    with open(os.path.join(dataset_to_process['save_path'], dataset_to_process['parameters_folder'], config_files[0]), 'w') as fp:
        json.dump(dataset_to_process['par_set'], fp)
elif dataset_to_process['registration_details']['algorithm'].lower() == 'elastix':
    # Elastix
    if DEBUGMODE:
        print(f'Define the parameters for the registration. Please wait...')
    dataset_to_process['par_set'] = itk.ParameterObject.New()
    for par_files in sorted(dataset_to_process['registration_details']['configuration_files']):
        dataset_to_process['par_set'].AddParameterFile(os.path.join(configpath, par_files))
        # Copy the parameters files to the output folder:
        shutil.copy2(os.path.join(configpath, par_files), os.path.join(dataset_to_process['save_path'],
                                                                       dataset_to_process['parameters_folder'],
                                                                       par_files))

    if DEBUGMODE:
        print('Parameters for ELASTIX:')
        print(dataset_to_process['par_set'])
else:
    print(f"Registration algorithm {dataset_to_process['registration_details']['algorithm']} not yet implemented. Please try again with a different option")

Define the parameters for the registration. Please wait...


AttributeError: module 'itk' has no attribute 'ParameterObject'

 ```
 Loop over the patientes to run the registration --> list(dataset_to_process['datasets'].keys())
 The workflow inside the loop is as follows:
  Loop over the visits  --> list(dataset_to_process['datasets'][list(dataset_to_process['datasets'].keys())[i]]['visits'].keys())
   Loads the Nifti file asigned to the fixed volume
     Initialises the 4D volume
     Loops over the moving dataset --> dataset_to_process['datasets'][list(dataset_to_process['datasets'].keys())[i]]['visits'][list(dataset_to_process['datasets'][list(dataset_to_process['datasets'].keys())[j]]['visits'].keys())[z]]['path2moving']
        Loads the Nifti file
        if moving_index == fixed_index:
            skip registration and assigns the output to aux
        else:
            runs registration
        saves the output data
        concatenate to 4D volume
```     

In [ ]:
print(f'Time at start: {time.ctime()}')
init_time = time.perf_counter()
for patient in dataset_to_process['datasets'].keys():
    start_registering_patient = time.perf_counter()
    for patient_visit in dataset_to_process['datasets'][patient]['visits']:
        start_registering_visit = time.perf_counter()
        patient_visit_outputpath = os.path.join(dataset_to_process['data_path'], patient, dataset_to_process['datasets'][patient]['visits'][patient_visit]['path'])
        print(patient_visit_outputpath)
        path2fixed = dataset_to_process['datasets'][patient]['visits'][patient_visit]['path2fixed']
        print(f'Fixed Volume: {path2fixed}')
        print(''.join(['-']*100))
        fixed_volume = itk.imread(path2fixed)
        fixed_voume_type = type(fixed_volume)
        if bias_correction is not None:
            # Apply the N4ITK correction to the fixed image
            print('Applying N4ITK correction to the fixed volume...')
            # fixed_volume_small = itk.shrink_image_filter(fixed_volume, shrink_factors=[2.0] * fixed_volume.GetImageDimension())
            # corrector = itk.N4BiasFieldCorrectionImageFilter.New(fixed_volume_small)
            # corrector.Update()
            fixed_volume_SS3 = itk.N4BiasFieldCorrectionImageFilter(fixed_volume)
            cast_filter_fixed_volume  = itk.CastImageFilter[type(fixed_volume_SS3), fixed_voume_type].New()
            cast_filter_fixed_volume.SetInput(fixed_volume_SS3)
            fixed_volume = cast_filter_fixed_volume.GetOutput()
            # fixed_volume.Update()
            # log_bias_field_fixed = fixed_volume.ReconstructBiasField(corrector.GetLogBiasFieldControlPointLattice())
            # itk.imwrite(log_bias_field, moving_set.replace(dataset_to_process['study_path'], dataset_to_process['data_path']))


        if dataset_to_process['registration_details']['algorithm'].lower() == 'ants':
            # Convert ITK image to ANTs:
            fixed_volume_ants = itk2ants(fixed_volume)
            # Test whether I need to separate the Bias Correction and use the one implemented in ANTsPy
            # https://antspy.readthedocs.io/en/latest/utils.html#ants.n4_bias_field_correction
            # ouput = ants.n4_bias_field_correction(input)

        # Get the list of moving datasets:
        moving_datasets = dataset_to_process['datasets'][patient]['visits'][patient_visit]['path2moving']

        # ITK concatenation output: Defines the 4D volume from this fixed image
        # To stack the volumes, use the function TileFilter, following the example at
        # https://examples.itk.org/src/filtering/imagegrid/create3dvolume/documentation
        # However, as I found out the hard way, the method SetInput works in order, even if it is within a loop, it works lexicographically (i.e. ordinal numbers)
        # so we'll recycle the list required for ANTs and after the registration, populates the tile in another (much quicker) loop
        input_dimension = fixed_volume.GetImageDimension()
        pixel_type = itk.template(fixed_volume)[1][0]
        output_dimension = input_dimension + 1
        input_image_type = itk.Image[pixel_type, input_dimension]
        output_image_type = itk.Image[pixel_type, output_dimension]
        layout = [1, 1, 1, len(moving_datasets)]
        registered_tiles = itk.TileImageFilter[input_image_type, output_image_type].New()
        registered_tiles.SetLayout(layout)


        for idx, moving_set in enumerate(moving_datasets):
            # Create the sub-folder in the output directories:
            os.makedirs(os.path.join(patient_visit_outputpath, str(idx+1)), exist_ok=True)
            # If the index is the Fixed Volume, just copy it to the output folder:
            if ((idx+1) == fixed_volume_pos) and (not register_fixed):
                if (bias_correction):
                    # If bias field was applied, then we have to save the corrected image, instead of just copying the raw
                    itk.imwrite(fixed_volume, path2fixed.replace(dataset_to_process['study_path'], dataset_to_process['data_path']))
                else:
                    print(f'No registration needed, {moving_set} is the fixed volume')
                    shutil.copy2(path2fixed, path2fixed.replace(dataset_to_process['study_path'], dataset_to_process['data_path']))
                registered_tiles.SetInput(idx, fixed_volume)
            else:
                # Any other case, just run the registration algorithm
                print(f'Registering dataset {moving_set} to reference volume {path2fixed}. Please wait...')
                start_registration_run = time.perf_counter()
                moving_volume = itk.imread(moving_set)
                moving_volume_type = type(moving_volume)
                if bias_correction is not None:
                    # Apply the N4ITK correction to the moving image
                    print('Applying N4ITK correction to the moving volume...')
                    # moving_volume_small = itk.shrink_image_filter(moving_volume, shrink_factors=[2.0] * moving_volume.GetImageDimension())
                    # mov_corrector = itk.N4BiasFieldCorrectionImageFilter.New(moving_volume_small)
                    # mov_corrector.Update()
                    moving_volume_SS3 = itk.N4BiasFieldCorrectionImageFilter(moving_volume)
                    cast_filter_moving_volume  = itk.CastImageFilter[type(moving_volume_SS3), moving_volume_type].New()
                    cast_filter_moving_volume.SetInput(moving_volume_SS3)
                    # cast_filter_moving_volume.Update()
                    moving_volume = cast_filter_moving_volume.GetOutput()
                    # log_bias_field_moving = fixed_volume.ReconstructBiasField(mov_corrector.GetLogBiasFieldControlPointLattice())
                if dataset_to_process['registration_details']['algorithm'].lower() == 'elastix':
                    if histogram_matching: # ANTsPy has this as default (see comment above)
                        # Apply histogram matching between fixed and moving
                        print('Applying histogram matching between fixed and moving images...')
                        moving_volume = itk.HistogramMatchingImageFilter(moving_volume, fixed_volume)
                    print('Running registration algorithm...')
                    warped_moving, result_transform_pars = itk.elastix_registration_method(fixed_volume ,
                                                                                           moving_volume,
                                                                                           parameter_object=dataset_to_process['par_set'],
                                                                                           log_to_console=False)
                elif dataset_to_process['registration_details']['algorithm'].lower() == 'ants':
                    print('Running registration algorithm...')
                    moving_volume_ants = itk2ants(moving_volume)
                    if histogram_matching:
                        print('Applying histogram matching between fixed and moving images...')
                        moving_volume_ants = ants.histogram_match_image(moving_volume_ants, fixed_volume_ants)
                    registeredOutput = ants.registration(fixed=fixed_volume_ants , moving=moving_volume_ants, **dataset_to_process['par_set'])
                    warped_moving = ants2itk(registeredOutput['warpedmovout'])


                end_registration_run = time.perf_counter()
                elp_registration_single = end_registration_run - start_registration_run
                print(f'Elapsed time to register single volume (incl. loading the data): {elp_registration_single:0.2f}[s] ({timedelta(seconds=elp_registration_single)})')
                print(f'Adding registered volume to the 4D tile...')
                registered_tiles.SetInput(idx, warped_moving)
                print(f"Saving the output result in {moving_set.replace(dataset_to_process['study_path'], dataset_to_process['data_path'])}")
                itk.imwrite(warped_moving, moving_set.replace(dataset_to_process['study_path'], dataset_to_process['data_path']))
                print(f'Finished registering timepoint {idx+1}')
            print(''.join(['=']*100))
        end_registering_visit = time.perf_counter()
        elp_registration_visit = end_registering_visit - start_registering_visit
        print(f'Elapsed time to register all timepoints in a visit (incl. loading the data): {elp_registration_visit:0.2f}[s] ({timedelta(seconds=elp_registration_visit)})')

        if BATCHMODE:
            # When running in BatchMode adds a small pause to avoid the "IOStream.flush timed out" error
            print('Just breathing a little...')
            time.sleep(5)
            print('Ready to continue!')

        # Saving the 4D time series :
        start_saving_4Dvol = time.perf_counter()
        reg_writer = itk.ImageFileWriter[output_image_type].New()
        reg_writer.SetFileName(os.path.join(os.path.split(patient_visit_outputpath)[0],
                                            '.'.join([os.path.split(dataset_to_process['datasets'][patient]['visits'][patient_visit]['path'])[0],
                                                      'nii.gz'])))
        reg_writer.SetInput(registered_tiles.GetOutput())
        reg_writer.Update()

        end_saving_4Dvol = time.perf_counter()
        elp_saving_4Dvol = end_saving_4Dvol - start_saving_4Dvol
        print(f'Elapsed time to save the 4D volume: {elp_saving_4Dvol:0.2f}[s] ({timedelta(seconds=elp_saving_4Dvol)})')

        print(''.join(['*']*100))
    end_registering_patient = time.perf_counter()
    elp_registration_patient = end_registering_patient - start_registering_patient
    print(f'Elapsed time to register a whole patient dataset: {elp_registration_patient:0.2f}[s] ({timedelta(seconds=elp_registration_patient)})')
    print(''.join(['§']*100))

final_time = time.perf_counter()
elp_whole_loop = final_time - init_time
print(f'Elapsed time to register the complete test {TEST_NRO} data folder: {elp_whole_loop:0.2f}[s] ({timedelta(seconds=elp_whole_loop)})')
print(f'Time at the end: {time.ctime()}')

Time at start: Thu Mar 28 11:40:38 2024
/Users/joseulloa/Data/fMRIBreastData/tests/Test014/datasets/DC-ANON97378/DC-Pre-Treatment-20230621/301
Fixed Volume: /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/2/301_dyn_ethrive.nii.gz
----------------------------------------------------------------------------------------------------
Registering dataset /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/1/301_dyn_ethrive.nii.gz to reference volume /Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/2/301_dyn_ethrive.nii.gz. Please wait...
Running registration algorithm...
Applying histogram matching between fixed and moving images...
Elapsed time to register single volume (incl. loading the data): 104.37[s] (0:01:44.365272)
Adding registered volume to the 4D tile...
Saving the output result in /Users/joseulloa/Data/fMRIBreastData/tests/Test014/datasets/DC-ANON97378/DC-Pre-T

In [ ]:
fixed_volume = itk.imread('/Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/1/301_dyn_ethrive.nii.gz')
fixed_volume_2 = itk.imread('/Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/2/301_dyn_ethrive.nii.gz')
moving_volume = itk.imread('/Users/joseulloa/Data/fMRIBreastData/NiftiData/DC-ANON97378/DC-Pre-Treatment-20230621/301/5/301_dyn_ethrive.nii.gz')

itk.imwrite(fixed_volume, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_fixed_001.nii.gz'))
itk.imwrite(fixed_volume_2, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_fixed_002.nii.gz'))
itk.imwrite(moving_volume, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_moving_005.nii.gz'))

bias_corrected_fixed_volume = itk.N4BiasFieldCorrectionImageFilter(fixed_volume)
bias_corrected_fixed_volume_2 = itk.N4BiasFieldCorrectionImageFilter(fixed_volume_2)
bias_corrected_moving_volume = itk.N4BiasFieldCorrectionImageFilter(moving_volume)

In [ ]:
histo_matched_005to001 = itk.HistogramMatchingImageFilter(fixed_volume, moving_volume)
histo_matched_005to002 = itk.HistogramMatchingImageFilter(fixed_volume_2, moving_volume)
itk.imwrite(histo_matched_005to001, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_histo_match_005to001.nii.gz'))
itk.imwrite(histo_matched_005to002, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_histo_match_005to002.nii.gz'))

itk.imwrite(bias_corrected_fixed_volume, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_bias_corrected_fixed_volume.nii.gz'))
itk.imwrite(bias_corrected_fixed_volume_2, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_bias_corrected_fixed_volume_2.nii.gz'))
itk.imwrite(bias_corrected_moving_volume, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_bias_corrected_moving_volume.nii.gz'))



In [ ]:
# To give more flexibility to fine-tune the parameters:
small_image = itk.shrink_image_filter(fixed_volume, shrink_factors=[2.0] * fixed_volume.GetImageDimension())

corrector = itk.N4BiasFieldCorrectionImageFilter.New(small_image)
# corrector.SetNumberOfFittingLevels(num_fitting_levels)
# corrector.SetMaximumNumberOfIterations([num_iterations] * num_fitting_levels)
corrector.Update()

full_res_corrector = itk.N4BiasFieldCorrectionImageFilter.New(fixed_volume)
log_bias_field = full_res_corrector.ReconstructBiasField(corrector.GetLogBiasFieldControlPointLattice())

itk.imwrite(full_res_corrector, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_bias_full_res_corrected_fixed_volume.nii.gz'))
itk.imwrite(log_bias_field, os.path.join('/Users/joseulloa/Data/fMRIBreastData/', 'aux', 'DC-Pre-Treatment-20230621_bias_field_fixed_volume.nii.gz'))



In [ ]:
new_fixed_imge = itk.N4BiasFieldCorrectionImageFilter(fixed_volume)

AttributeError: 'itkImageSS3' object has no attribute 'ImageType'

In [ ]:
fixed_volume = itk.imread(path2fixed)
fixed_volume_SS3 = itk.N4BiasFieldCorrectionImageFilter(fixed_volume)
fixed_volume  = itk.CastImageFilter[fixed_volume_SS3, fixed_volume].New()
fixed_volume.SetInput(fixed_volume_SS3)
# fixed_volume.Update()
fixed_volume

<itk.itkCastImageFilterPython.itkCastImageFilterISS3IF3; proxy of <Swig Object of type 'itkCastImageFilterISS3IF3 *' at 0x472d55f80> >

In [ ]:
see_output = fixed_volume.GetOutput()
see_output

<itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x472ef54d0> >

In [ ]:

itk.HistogramMatchingImageFilter(fixed_volume, new_output)

<itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x420ff8630> >